In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import ot
import sys
import scanpy as sc
import anndata as ad
import os
import tarfile
import scipy.io
import h5py
from scipy.sparse import issparse
from scipy.sparse import csr_matrix

In [23]:
adata = ad.read_h5ad('/workspace/ImputationOT/data/facs_normal_lung_blood_scanpy.20200205.RC4.h5ad')
print(adata)

AnnData object with n_obs × n_vars = 9409 × 60970
    obs: 'nGene', 'nReads', 'orig.ident', 'plate.barcode', 'cell.id', 'organism', 'tissue', 'region', 'label', 'sorter', 'sort.location', 'sample', 'location', 'patient', 'percent.ercc', 'percent.ribo', 'gating', 'free_annotation', 'Number of input reads', 'Uniquely mapped reads number', 'Number of splices: Total', 'Number of splices: Annotated (sjdb)', 'Number of splices: GT/AG', 'Number of splices: GC/AG', 'Number of splices: AT/AC', 'Number of splices: Non-canonical', 'Number of reads mapped to multiple loci', 'Number of reads mapped to too many loci', 'Number of chimeric reads', 'Mapping speed, Million of reads per hour', 'Average input read length', 'Uniquely mapped reads %', 'Average mapped length', 'Mismatch rate per base, %', 'Deletion rate per base', 'Deletion average length', 'Insertion rate per base', 'Insertion average length', '% of reads mapped to multiple loci', '% of reads mapped to too many loci', '% of reads unmapped: 

In [42]:
print(adata.obs['free_annotation'])

index
A1_B002014.gencode.vH29           Capillary Aerocyte_P2
A1_B003138.gencode.vH29           Capillary Aerocyte_P2
A1_B003140.gencode.vH29     Capillary Intermediate 1_P2
A10_B003138.gencode.vH29    Capillary Intermediate 1_P2
A12_B003141.gencode.vH29          Capillary Aerocyte_P2
                                       ...             
M5_B001771.gencode.vH29                     Ciliated_P3
N2_B001769.gencode.vH29                     Ciliated_P3
N2_B002460.gencode.vH29                     Ciliated_P3
O2_B001774.gencode.vH29                     Ciliated_P3
O7_B001774.gencode.vH29                     Ciliated_P3
Name: free_annotation, Length: 9409, dtype: category
Categories (95, object): ['Adventitial Fibroblast_P1', 'Adventitial Fibroblast_P2', 'Adventitial Fibroblast_P3', 'Airway Smooth Muscle_P1', ..., 'Vascular Smooth Muscle_P1', 'Vascular Smooth Muscle_P2', 'Vascular Smooth Muscle_P3', 'Vein_P2']


In [40]:
print(adata.obs['sort.location'].value_counts())

sort.location
biohub     7015
beckman    1642
sim1        752
Name: count, dtype: int64


In [29]:
X = adata.X
print("Matrix Shape:", X.shape)
print("Density:", np.count_nonzero(X) / (X.shape[0] * X.shape[1]))
# print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

Matrix Shape: (9409, 60970)
Density: 0.04190173273600859
Minimum Value: 0.0
Maximum Value: 13.540438


In [4]:
batch_counts = adata.obs['Batch'].value_counts()
print(batch_counts)

Batch
1    13086
8    12745
7    11752
6    11170
5    11017
4    10553
9    10315
2     8618
3     8243
Name: count, dtype: int64


In [17]:
print(len(adata.obs['Annotation'].unique()))
print(adata.obs['Annotation'])

28
RV8919578_AAACCTGAGAAACCTA-1    NK CD56(bright)
RV8919578_AAACCTGAGAGCAATT-1            Naïve B
RV8919578_AAACCTGAGAGCTGGT-1                pDC
RV8919578_AAACCTGAGCGAAGGG-1       NK CD56(dim)
RV8919578_AAACCTGAGCGATGAC-1    NK CD56(bright)
                                     ...       
RV8959686_TTTGTCAGTATGAATG-1        CD4 naïve T
RV8959686_TTTGTCAGTCGGCATC-1            Naïve B
RV8959686_TTTGTCAGTGACAAAT-1    NK CD56(bright)
RV8959686_TTTGTCAGTTGCCTCT-1        CD4 naïve T
RV8959686_TTTGTCATCAGTCCCT-1          Platelets
Name: Annotation, Length: 97499, dtype: category
Categories (28, object): ['CD4 memory T', 'CD4 naïve T', 'CD8 memory T', 'CD8 naïve T', ..., 'cDC2', 'cDC3', 'pDC', 'γδT']


In [16]:
print(len(adata.obs['Batch'].unique()))
print(adata.obs['Batch'])

9
RV8919578_AAACCTGAGAAACCTA-1    1
RV8919578_AAACCTGAGAGCAATT-1    1
RV8919578_AAACCTGAGAGCTGGT-1    1
RV8919578_AAACCTGAGCGAAGGG-1    1
RV8919578_AAACCTGAGCGATGAC-1    1
                               ..
RV8959686_TTTGTCAGTATGAATG-1    9
RV8959686_TTTGTCAGTCGGCATC-1    9
RV8959686_TTTGTCAGTGACAAAT-1    9
RV8959686_TTTGTCAGTTGCCTCT-1    9
RV8959686_TTTGTCATCAGTCCCT-1    9
Name: Batch, Length: 97499, dtype: category
Categories (9, object): ['1', '2', '3', '4', ..., '6', '7', '8', '9']
